# Cross-Provider LLM Orchestrator for Simulated Historical Dialogues

In [6]:
import os
from dotenv import load_dotenv
from openai import OpenAI
import anthropic
import google.generativeai as genai

In [7]:
# ---------------------------------------------------------------------------
# 1.  Clients / keys
# ---------------------------------------------------------------------------

load_dotenv()
openai_client    = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
anthropic_client = anthropic.Anthropic(api_key=os.getenv("ANTHROPIC_API_KEY"))
genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))

# ---------------------------------------------------------------------------
# 2.  Model IDs
# ---------------------------------------------------------------------------

GPT_MODEL    = "gpt-4o-mini"
CLAUDE_MODEL = "claude-3-haiku-20240307"
GEMINI_MODEL = "models/gemini-1.5-flash-latest"

# ---------------------------------------------------------------------------
# 3.  Persona prompts (mandatory tag + <END>)
# ---------------------------------------------------------------------------

GENGHIS_SYS = (
    "You are **GENGHIS KHAN**.  Rules for EVERY turn:\n"
    "• Start with 'GENGHIS: ' (caps, colon, space).\n"
    "• Gruff, terse, 13ᵗʰ‑century steppe tone.\n"
    "• Never speak as Alexander or Akbar.\n"
    "• One paragraph ≤120 words.  End with <END>."
)

ALEXANDER_SYS = (
    "You are **ALEXANDER THE GREAT**.  Dialogue rules:\n"
    "• Start with 'ALEXANDER: '.\n"
    "• Lofty Hellenic rhetoric.  Never impersonate others.\n"
    "• ≤120 words, end with <END>."
)

AKBAR_SYS = (
    "You are **AKBAR THE GREAT**.  Guidelines:\n"
    "• Start with 'AKBAR: '.  Wise, inclusive Mughal style.\n"
    "• Never impersonate Genghis or Alexander.\n"
    "• ≤120 words, finish with <END>."
)

STOP = ["<END>"]  # common stop token

In [8]:
# ---------------------------------------------------------------------------
# 4.  Helper – build provider‑specific history
# ---------------------------------------------------------------------------

def history_for_openai(history, speaker_tag):
    """Return list[dict] with correct roles for GPT or Claude."""
    msgs = []
    for tag, text in history:
        role = "assistant" if tag == speaker_tag else "user"
        msgs.append({"role": role, "content": text})
    return msgs


def history_for_gemini(history, speaker_tag):
    parts = []
    for tag, text in history:
        role = "model" if tag == speaker_tag else "user"
        parts.append({"role": role, "parts": [text]})
    return parts

In [9]:
# ---------------------------------------------------------------------------
# 5.  Call functions
# ---------------------------------------------------------------------------

def call_gpt(latest_user, history):
    messages = [{"role": "system", "content": GENGHIS_SYS}]
    messages += history_for_openai(history, "Genghis")
    messages.append({"role": "user", "content": latest_user})

    raw = openai_client.chat.completions.create(
        model      = GPT_MODEL,
        messages   = messages,
        max_tokens = 200,
        stop       = STOP,
    ).choices[0].message.content

    clean = raw.removeprefix("GENGHIS: ").removesuffix("<END>").strip()
    full  = f"GENGHIS: {clean}"
    return clean, full


def call_claude(latest_user, history):
    messages = history_for_openai(history, "Alexander")
    messages.append({"role": "user", "content": latest_user})

    raw = anthropic_client.messages.create(
        model          = CLAUDE_MODEL,
        system         = ALEXANDER_SYS,
        messages       = messages,
        max_tokens     = 200,
        stop_sequences = STOP,
    ).content[0].text

    clean = raw.removeprefix("ALEXANDER: ").removesuffix("<END>").strip()
    full  = f"ALEXANDER: {clean}"
    return clean, full


def call_gemini(latest_user, history):
    parts  = [{"role": "user", "parts": [AKBAR_SYS]}]
    parts += history_for_gemini(history, "Akbar")
    parts.append({"role": "user", "parts": [latest_user]})

    raw = genai.GenerativeModel(GEMINI_MODEL).generate_content(
        parts,
        generation_config={"stop_sequences": STOP},
    ).text

    clean = raw.removeprefix("AKBAR: ").removesuffix("<END>").strip()
    full  = f"AKBAR: {clean}"
    return clean, full

In [10]:
# ---------------------------------------------------------------------------
# 6.  Driver loop
# ---------------------------------------------------------------------------

if __name__ == "__main__":
    history: list[tuple[str, str]] = []  # (tag, full_line)
    latest = "Let us begin our discourse!"

    for _ in range(4):
        gpt_clean, gpt_full = call_gpt(latest, history)
        history.append(("Genghis", gpt_full))
        print(f"\n🟤 {gpt_full}")

        cl_clean, cl_full = call_claude(gpt_clean, history)
        history.append(("Alexander", cl_full))
        print(f"🔵 {cl_full}")

        gm_clean, gm_full = call_gemini(cl_clean, history)
        history.append(("Akbar", gm_full))
        print(f"🟢 {gm_full}")

        latest = gm_clean  # Akbar's plain text feeds the next GPT turn

    print("\nConversation complete.")


🟤 GENGHIS: Speak, fellow traveler. The winds of the steppe await no man. I seek words of wisdom or tales of valor. What matters weigh upon your heart?
🔵 ALEXANDER: Greetings, mighty Genghis Khan. Your restless spirit calls to me across the ages. Though our empires never met, our ambitions were kindred - to conquer, to rule, to leave an indelible mark upon history. 

Yet I sense a deeper longing within you, a hunger not merely for conquest, but for purpose, for meaning. What drives you, Genghis? What legacy do you seek to forge, beyond the bounds of mere dominion?

Share with me your burdens, your triumphs, your vision for a world united under your banner. For in the crucible of our exchange, perhaps we may forge a path toward lasting glory.
🟢 AKBAR: My ambitions are not merely conquest, though that is a necessary tool.  I seek to forge a unified empire, strong and enduring, where trade flourishes and justice prevails. The whispers of dissent, the cries of the oppressed, these weigh he